In [33]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
# import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

ImportError: Traceback (most recent call last):
  File "C:\Users\arted\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Произошел сбой в программе инициализации библиотеки динамической компоновки (DLL).


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [14]:
df = pd.read_csv('data.csv')
df["Квартиры₽/м²"] = df["Квартиры₽/м²"].str.replace(" ", "", regex=True)

data = df['Квартиры₽/м²']

data = data.astype(int)  # Преобразование цен в int

In [18]:
train_size = int(len(data) * 0.9)
train, test = data[:train_size], data[train_size:]

In [19]:
model_arima = ARIMA(train, order=(5,1,0))
model_arima_fit = model_arima.fit()
predictions_arima = model_arima_fit.forecast(steps=len(test))

In [21]:
mae_arima = mean_absolute_error(test, predictions_arima)
rmse_arima = np.sqrt(mean_squared_error(test, predictions_arima))
print(f'ARIMA: MAE={mae_arima:.2f}, RMSE={rmse_arima:.2f}')

ARIMA: MAE=22849.26, RMSE=23636.93


In [22]:
scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = scaler.fit_transform(data.values.reshape(-1,1))

In [23]:
def create_dataset(dataset, look_back=10):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

In [24]:
look_back = 10
X_train, y_train = create_dataset(data_scaled[:train_size], look_back)
X_test, y_test = create_dataset(data_scaled[train_size:], look_back)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [25]:
model_1 = keras.Sequential([
    LSTM(50, return_sequences=True, input_shape=(look_back, 1)),
    LSTM(50),
    Dense(1)
])
model_1.compile(optimizer='adam', loss='mean_squared_error')
model_1.fit(X_train, y_train, epochs=20, batch_size=1, verbose=1)

NameError: name 'Sequential' is not defined

In [34]:
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "C:\Users\arted\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Произошел сбой в программе инициализации библиотеки динамической компоновки (DLL).


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.